In [ ]:
import sqlite3
import pandas as pd
import plotly.express as px
from pandasql import sqldf, load_meat, load_births
from pysqldf import SQLDF
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
from bokeh.models import Legend, LegendItem
from bokeh.palettes import Category20, Colorblind, Spectral11, Spectral4
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, FactorRange, ColorBar, LinearColorMapper, Title
from bokeh.io import show, output_notebook, export_svg
from bokeh.transform import factor_cmap, transform
from sklearn.preprocessing import MinMaxScaler
from plotly.subplots import make_subplots
from bokeh.layouts import gridplot
from bokeh.core.properties import value

# Set the display.max_rows option to None to display all rows
pd.set_option('display.max_rows', None)

# Set the display.max_columns option to None to display all columns
pd.set_option('display.max_columns', None)

def matplotlib_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []
    
    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k*h)[:3])*255))
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])
        
    return pl_colorscale

def split_and_insert_break(organism):
    if len(organism) >11:
        words = organism.split(' ')
        for i in range(len(words)):
            words[i] = words[i] + '<br>'
            organism = ' '.join(words)
    return organism 



def plot_dict_of_dfs(dfs):
    output_notebook()
    plots = []
    for key, df in dfs.items():
        for col in df.columns:
            print(col)
            print()
            if pd.api.types.is_numeric_dtype(df[col]):
                hist, edges = np.histogram(df[col].dropna(), bins=50)
                p = figure(title=f"Histogram of {col}", tools='', background_fill_color="#fafafa")
                p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], fill_color="navy", line_color="white", alpha=0.5)
                p.y_range.start = 0
                p.xaxis.axis_label = 'Value'
                p.yaxis.axis_label = 'Frequency'
                p.grid.grid_line_color="white"
#             elif col == 'score':
#                 continue
            else:
                data = df[col].value_counts()
                source = ColumnDataSource(data=dict(x=data.index, y=data.values))
                p = figure(x_range=data.index.tolist(), plot_height=300, toolbar_location=None, title=f"Counts of {col}")
                p.vbar(x='x', top='y', width=0.9, source=source)

            plots.append(p)

    grid = gridplot(plots, ncols=3)

    show(grid)

In [ ]:
palette = ['#4169e1',
'#e9967a',
'#dc143c',
'#00ffff',
'#00bfff',
'#f4a460',
'#9370db',
'#0000ff',
'#ff6347',
'#d8bfd8',
'#ff00ff',
'#db7093',
'#f0e68c',
'#ffff54',
'#6495ed',
'#dda0dd',
'#87ceeb',
'#bc8f8f',
'#663399',
'#008080',
'#b8860b',
'#bdb76b',
'#4682b4',
'#000080',
'#d2691e',
'#9acd32',
'#20b2aa',
'#cd5c5c',
'#32cd32',
'#8fbc8f',
'#8b008b',
'#b03060',
'#d2b48c',
'#66cdaa',
'#a9a9a9',
'#dcdcdc',
'#2f4f4f',
'#556b2f',
'#8b4513',
'#6b8e23',
'#2e8b57',
'#191970',
'#708090',
'#8b0000',
'#483d8b',
'#008000',
'#ff4500',
'#ffa500',
'#ffd700',
'#c71585',
'#0000cd',
'#7cfc00',
'#00ff00',
'#9400d3',
'#ba55d3',
'#00fa9a',
'#ff1493',
'#afeeee',
'#ee82ee',
'#98fb98',
'#7fffd4',
'#ff69b4',
'#ffe4c4',
'#ffb6c1']

In [ ]:
conn = sqlite3.connect('mprabase_v4_9.2.db')
cur = conn.cursor()
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()

tablesofdatabase = {}
for table in tables:
    query = f"SELECT * from {table[0]}"
    
    tablesofdatabase[table[0]] = pd.read_sql_query(query, conn)


In [ ]:
pysqldf = lambda q: sqldf(q, globals())

counts = {}
for key in tablesofdatabase.keys():
    locals()[key] = tablesofdatabase[key]
    query = f"SELECT COUNT(*) FROM {key};"
    result = pysqldf(query)
    counts[key] = result.values[0][0]

In [ ]:
columnlists={}
print("------------Columns------------")
print()
for i in tablesofdatabase:
    columnlists[i]=tablesofdatabase[i].columns.tolist()
    print(i, ':   ', columnlists[i])

print()
print("------------Columns------------")


In [ ]:
summarydata = pd.DataFrame(counts, index=['count'])
summarydata

In [ ]:
tablesofdatabase['designed_library']

In [ ]:
len(tablesofdatabase['designed_library']['datasets_id'].unique())

In [ ]:
len(tablesofdatabase['designed_library']['library_id'].unique())

In [ ]:
tablesofdatabase['library_sequence'].head()

In [ ]:
len(tablesofdatabase['library_sequence']['library_id'].unique())

In [ ]:
tablesofdatabase['element_score'].head()

In [ ]:
len(tablesofdatabase['element_score']['sample_id'].unique())

In [ ]:
tablesofdatabase['element_rep_score'].head()

In [ ]:
len(tablesofdatabase['element_rep_score']['sample_id'].unique())

In [ ]:
tablesofdatabase['datasets'].head() 

In [ ]:
len(tablesofdatabase['datasets']['datasets_id'].unique())

In [ ]:
tablesofdatabase['sample'].head()

In [ ]:
len(tablesofdatabase['sample']['library_id'].unique())

In [ ]:
merged_df = pd.merge(tablesofdatabase['designed_library'], tablesofdatabase['datasets'], on='datasets_id', how='outer')
final_df = pd.merge(merged_df, tablesofdatabase['sample'], on='library_id', how='outer')

In [ ]:
final_df['Library_strategy'] = final_df['Library_strategy'].str.replace('\n', '')

In [ ]:
grouped = final_df.groupby(['Organism','Library_strategy','Cell_line_tissue']).describe()['sample_id']['count']

In [ ]:
#https://plotly.com/python/sunburst-charts/?_ga=2.93626589.1720091979.1695947763-1327420249.1695947763

grouped2 = pd.DataFrame(grouped.groupby(['Organism', 'Library_strategy','Cell_line_tissue']).sum())
total = grouped2.groupby(['Organism'])['count'].sum().sum()
total_per_organism = grouped2.groupby(['Organism'])['count'].sum()
total_per_organism_library = grouped2.groupby(['Organism','Library_strategy'])['count'].sum()
total_per_organism_library_tissue = pd.DataFrame(grouped2.groupby(['Organism','Library_strategy','Cell_line_tissue'])['count'].sum())

percentageoforganism =grouped2.groupby(['Organism'])['count'].apply(lambda x: x.sum() / total *100)
librarypercentageoforganism = grouped2.groupby(['Organism', 'Library_strategy'])['count'].apply(lambda x: (x.sum() / total_per_organism[x.name[0]]) * 100)
librarypercentageoforganism = librarypercentageoforganism.groupby(level=[0,1]).sum()  / 100 * percentageoforganism
tissuepercentageoflibraryperorganism = pd.DataFrame(grouped2.groupby(['Organism','Library_strategy'])['count'].apply(lambda x: x/x.sum()*100))
unique_index = pd.Index(tissuepercentageoflibraryperorganism.index).unique()
new_df = pd.DataFrame(index=unique_index, data=tissuepercentageoflibraryperorganism['count']).reset_index()
new_index = [(x[0], x[1], x[-1]) for x in new_df['index']]
new_df['index'] = new_index
new_df.index = new_df['index']
grouped2['percentage'] = new_df['count']
grouped2['percentage'] = grouped2['percentage']  / 100 * librarypercentageoforganism
df = grouped2.reset_index()


fig = px.sunburst(df, path=['Organism', 'Library_strategy', 'Cell_line_tissue'], values='percentage',color_discrete_sequence=Colorblind[len(grouped2.index.get_level_values(0).unique())])

layout = go.Layout(
                   showlegend=False,
                   autosize=False,
                   width=1000,
                   height=1000
                ) 

fig.update_layout(layout)
fig.update_traces(textfont_size=18)
fig.update_traces(textfont=dict(family="Arial"))

fig.write_image("piechart_MPRA.svg")

fig.show()


In [ ]:
df = final_df
df['labs'] = df['labs'].astype(str)
df['labs'] = df['labs'].apply(lambda x: ','.join(sorted([i.strip() for i in x.split(',')])))

In [ ]:
output_notebook()

grouped = df.groupby(['Organism','labs','Library_strategy']).size().reset_index(name='counts')
pivot_df = grouped.pivot_table(values='counts', index=['Organism', 'labs'], columns='Library_strategy', fill_value=0).reset_index()
library_strategies = pivot_df.columns[2:].tolist()
pivot_df = pivot_df.sort_values(by=['Organism',])
x = [(Organism, labs) for Organism, labs in zip(pivot_df['Organism'], pivot_df['labs'])]
pivot_df['x'] = x

source = ColumnDataSource(pivot_df)
p = figure(x_range=FactorRange(*x), height=1000, width=3000)
p.vbar_stack(library_strategies, x='x', width=0.475, color=[palette[i] for i,v in enumerate(library_strategies)], line_color='white',
             source=source, legend_label=[v for i,v in enumerate(library_strategies)])
p.legend.orientation = "vertical"
p.add_layout(p.legend[0], 'right')
p.legend.title= "MPRA Type"
p.y_range.start = 0
p.xaxis.major_label_orientation = 1.2
p.xgrid.grid_line_color = None
p.title.text_font_size = '20pt'
p.xaxis.axis_label_text_font_size = "25pt"
p.yaxis.axis_label_text_font_size = "20pt"
p.xaxis.major_label_text_font_size = "19pt"
p.yaxis.major_label_text_font_size = "17pt"
p.xaxis.group_text_font_size = "19pt"
p.legend.label_text_font_size = '14pt'
p.legend.title_text_font_size = '16pt'
p.yaxis.axis_label = "Count"
p.x_range = FactorRange(*x, factor_padding=-0.5, group_padding=1.5)
p.x_range.range_padding = 0

show(p)


In [ ]:
output_notebook()

grouped = df.groupby(['Organism','labs','Cell_line_tissue']).size().reset_index(name='counts')
pivot_df = grouped.pivot_table(values='counts', index=['Organism', 'labs'], columns='Cell_line_tissue', fill_value=0).reset_index()
library_strategies = pivot_df.columns[2:].tolist()
pivot_df = pivot_df.sort_values(by=['Organism',])
x = [(Organism, labs) for Organism, labs in zip(pivot_df['Organism'], pivot_df['labs'])]
pivot_df['x'] = x

source = ColumnDataSource(pivot_df)
p = figure(x_range=FactorRange(*x), height=1000, width=3000)
p.vbar_stack(library_strategies, x='x', width=0.475, color=[palette[i] for i,v in enumerate(library_strategies)], line_color='white',
             source=source, legend_label=[v for i,v in enumerate(library_strategies)])

p.legend.orientation = "vertical"
p.add_layout(p.legend[0], 'right')
p.legend.title= "Cell Type/Tissue"
p.y_range.start = 0
p.xaxis.major_label_orientation = 1.2
p.xgrid.grid_line_color = None
p.title.text_font_size = '20pt'
p.xaxis.axis_label_text_font_size = "25pt"
p.yaxis.axis_label_text_font_size = "20pt"
p.xaxis.major_label_text_font_size = "19pt"
p.yaxis.major_label_text_font_size = "17pt"
p.xaxis.group_text_font_size = "19pt"
p.legend.label_text_font_size = '14pt' 
p.legend.title_text_font_size = '16pt'
p.yaxis.axis_label = "Count"
p.x_range = FactorRange(*x, factor_padding=-0.5, group_padding=1.5)
p.x_range.range_padding = 0

show(p)


In [ ]:
output_notebook()

grouped = df.groupby(['Organism','Cell_line_tissue']).size().reset_index(name='counts')
pivot_df = grouped.pivot_table(values='counts', index=['Organism'], columns='Cell_line_tissue', fill_value=0).reset_index()
library_strategies = pivot_df.columns[1:].tolist()
pivot_df = pivot_df.sort_values(by=['Organism',])
x = [Organism for Organism in pivot_df['Organism']]
pivot_df['x'] = x

source = ColumnDataSource(pivot_df)
p = figure(x_range=FactorRange(*x), height=1000, width=800)
p.vbar_stack(library_strategies, x='x', width=0.475, color=[palette[i] for i,v in enumerate(library_strategies)], line_color='white',
             source=source, legend_label=[v for i,v in enumerate(library_strategies)])

p.legend.orientation = "vertical"
p.add_layout(p.legend[0], 'right')
p.legend.title= "Cell Line/Tissue"
p.y_range.start = 0
p.xaxis.major_label_orientation = 1.2
p.xgrid.grid_line_color = None
p.title.text_font_size = '20pt'
p.xaxis.axis_label_text_font_size = "25pt"
p.yaxis.axis_label_text_font_size = "20pt"
p.xaxis.major_label_text_font_size = "19pt"
p.yaxis.major_label_text_font_size = "17pt"
p.xaxis.group_text_font_size = "19pt"
p.legend.glyph_height = 15
p.legend.glyph_width = 10
p.legend.label_text_font_size = '12pt'
p.legend.title_text_font_size = '16pt' 
p.yaxis.axis_label = "Count"
p.x_range.range_padding = 0

show(p)


In [ ]:
output_notebook()

grouped = df.groupby(['Organism','Library_strategy']).size().reset_index(name='counts')
pivot_df = grouped.pivot_table(values='counts', index=['Organism'], columns='Library_strategy', fill_value=0).reset_index()
library_strategies = pivot_df.columns[1:].tolist()
pivot_df = pivot_df.sort_values(by=['Organism',])
x = [Organism for Organism in pivot_df['Organism']]
pivot_df['x'] = x

source = ColumnDataSource(pivot_df)
p = figure(x_range=FactorRange(*x), height=1000, width=800)
p.vbar_stack(library_strategies, x='x', width=0.475, color=[palette[i] for i,v in enumerate(library_strategies)], line_color='white',
             source=source, legend_label=[v for i,v in enumerate(library_strategies)])

p.legend.orientation = "vertical"
p.add_layout(p.legend[0], 'right')
p.legend.title= "MPRA Type"
p.legend.label_text_font_size = '13pt'
p.y_range.start = 0
p.xaxis.major_label_orientation = 1.2
p.xgrid.grid_line_color = None
p.title.text_font_size = '20pt'
p.xaxis.axis_label_text_font_size = "25pt"
p.yaxis.axis_label_text_font_size = "20pt"
p.xaxis.major_label_text_font_size = "19pt"
p.yaxis.major_label_text_font_size = "17pt"
p.xaxis.group_text_font_size = "19pt"
p.legend.glyph_height = 15
p.legend.glyph_width = 10
p.legend.label_text_font_size = '11pt'
p.legend.title_text_font_size = '16pt'
p.yaxis.axis_label = "Count"
p.x_range.range_padding = 0

show(p)


In [ ]:
order = ["Homo sapiens", "Mus musculus", "Escherichia coli", "Drosophila melanogaster"]
grouped = df.groupby(['Organism','Cell_line_tissue']).size().reset_index(name='counts')
organisms = grouped['Organism'].unique().tolist()
organisms = sorted(organisms, key=lambda x: order.index(x))
organism_cell_line_pairs = list(grouped[['Organism', 'Cell_line_tissue']].itertuples(index=False, name=None))
organism_cell_line_pairs.sort(key=lambda x: order.index(x[0]))
cell_lines_tissues = [pair[1] for pair in organism_cell_line_pairs]

data = {organism: {'counts': [], 'x': []} for organism in organisms}
for organism in organisms:
    for cell_line_tissue in cell_lines_tissues:
        count = grouped[(grouped['Organism'] == organism) & (grouped['Cell_line_tissue'] == cell_line_tissue)]['counts']
        count_value = count.iloc[0] if not count.empty else 0
        if count_value > 0:  
            data[organism]['x'].append(cell_line_tissue)
            data[organism]['counts'].append(count_value)

p = figure(x_range=cell_lines_tissues, height=500, width=1300, title="Cell Line/Tissue Counts by Organism")

vbars = []
colors = Spectral4
for i, organism in enumerate(organisms):
    source = ColumnDataSource(data=data[organism])
    vbar = p.vbar(x='x', top='counts', width=0.9, source=source, color=colors[i])
    vbars.append(vbar)

legend = Legend(items=[
    LegendItem(label=organism, renderers=[vbars[i]]) for i, organism in enumerate(organisms)
], location="top_center", orientation="horizontal")

p.add_layout(legend, 'above')
p.y_range.start = 0
p.xaxis.major_label_orientation = 1.57
p.xgrid.grid_line_color = None
p.title.text_font_size = '20pt'
p.xaxis.axis_label_text_font_size = "25pt"
p.yaxis.axis_label_text_font_size = "20pt"
p.xaxis.major_label_text_font_size = "19pt"
p.yaxis.major_label_text_font_size = "17pt"
p.xaxis.group_text_font_size = "19pt"
p.legend.label_text_font_size = '14pt' 
p.legend.title_text_font_size = '16pt' 
p.yaxis.axis_label = "Count"
p.x_range.range_padding = 0


export_svg(p, filename="celllinecountbyorganism_MPRA.svg")


show(p)


In [ ]:
grouped = df.groupby(['Organism','Library_strategy']).size().reset_index(name='counts')
organisms = grouped['Organism'].unique().tolist()
organisms = sorted(organisms, key=lambda x: order.index(x))
cell_lines_tissues = grouped['Library_strategy'].unique().tolist()

data = {'counts': [], 'x': []}
for organism in organisms:
    for cell_line_tissue in cell_lines_tissues:
        data['x'].append((organism, cell_line_tissue))
        count = grouped[(grouped['Organism'] == organism) & (grouped['Library_strategy'] == cell_line_tissue)]['counts']
        data['counts'].append(count.iloc[0] if not count.empty else 0)

source = ColumnDataSource(data=data)
p = figure(x_range=FactorRange(*data['x']), height=500, width=1300, title="MPRA Counts by Organism")
p.vbar(x='x', top='counts', width=0.9, source=source)

p.y_range.start = 0
p.xaxis.major_label_orientation = 1.56
p.xgrid.grid_line_color = None
p.title.text_font_size = '20pt'
p.xaxis.axis_label_text_font_size = "25pt"
p.yaxis.axis_label_text_font_size = "20pt"
p.xaxis.major_label_text_font_size = "19pt"
p.yaxis.major_label_text_font_size = "17pt"
p.xaxis.group_text_font_size = "19pt"
p.legend.label_text_font_size = '14pt' 
p.legend.title_text_font_size = '16pt' 
p.yaxis.axis_label = "Count"
p.x_range.range_padding = 0

export_svg(p, filename="mpracountbyorganism_MPRA.svg")

show(p)
